
# # AzureTranslateTool with AgentExecutor
# 

# This notebook demonstrates how to use the `AzureTranslateTool` in conjunction with an AgentExecutor. The agent will handle translation requests by utilizing the Azure OpenAI model and the `AzureTranslateTool`.


# ## Step 1: Import Required Libraries and Load Environment Variables
# We begin by importing the necessary libraries and loading the environment variables that include the Azure OpenAI and LangSmith API keys.


In [1]:
# Import necessary libraries
from libs.community.langchain_community.tools.azure_ai_services.text_translate import AzureAITextTranslateTool
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain import hub
import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env
load_dotenv()

True

In [3]:
# Load the prompt from LangSmith
LANGSMITH_KEY = os.getenv('LANGSMITH_KEY')
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

In [4]:
# Instantiate the AzureTranslateTool
tools = [AzureAITextTranslateTool()]

In [5]:
# Initialize the Azure OpenAI model
model = AzureChatOpenAI(
    openai_api_version=os.getenv('OPENAI_API_VERSION'),  
    azure_deployment=os.getenv('COMPLETIONS_MODEL'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT'),
    api_key=os.getenv('AZURE_API_KEY'),
    temperature=0.5,
    max_tokens=200,
    timeout=60,
    max_retries=10,
)

In [6]:
# Create the agent using the model, tools, and prompt
agent = create_structured_chat_agent(model, tools, prompt)
# Set up the AgentExecutor with verbose output
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

Example 1 - Simple Translation

In [7]:

# Input text to translate
input_text = "Hello, how are you?"# Invoke the agent with the translation request
try:
    response = agent_executor.invoke({"input": f"Can you translate the following text to te reo maori: {input_text}"})
    print("Translation result:", response)
except Exception as e:
    print(f"Error during translation: {e}")




> Entering new AgentExecutor chain...
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Hello, how are you?",
    "to_language": "mi"
  }
}Tēnā koe e pēhea ana koe?{
  "action": "Final Answer",
  "action_input": "Tēnā koe e pēhea ana koe?"
}

> Finished chain.
Translation result: {'input': 'Can you translate the following text to te reo maori: Hello, how are you?', 'output': 'Tēnā koe e pēhea ana koe?'}


Example 2 - Language Detection

In [8]:
# Example: Detect the language of "Bonjour tout le monde"
query_language_detection = {'input': 'Can you detect what language this phrase is in: Bonjour tout le monde'}

# Invoke the tool to detect language
try:
    response_language_detection = agent_executor.invoke(query_language_detection)
    print("Language detection result:", response_language_detection)
except Exception as e:
    print(f"Error during language detection: {e}")




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "The phrase 'Bonjour tout le monde' is in French."
}
```

> Finished chain.
Language detection result: {'input': 'Can you detect what language this phrase is in: Bonjour tout le monde', 'output': "The phrase 'Bonjour tout le monde' is in French."}


Example 3 Translate Complex Sentences

In [9]:
# Example: Translate a more complex sentence to French
query_complex_sentence = {'input': 'Can you translate this technical sentence to French: "Artificial intelligence will revolutionize healthcare in the next decade."'}

# Invoke the tool to translate the complex sentence
try:
    response_complex_sentence = agent_executor.invoke(query_complex_sentence)
    print("Complex translation result:", response_complex_sentence)
except Exception as e:
    print(f"Error during translation: {e}")




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Artificial intelligence will revolutionize healthcare in the next decade.",
    "to_language": "fr"
  }
}
```L’intelligence artificielle révolutionnera les soins de santé au cours de la prochaine décennie.Action:
```
{
  "action": "Final Answer",
  "action_input": "L’intelligence artificielle révolutionnera les soins de santé au cours de la prochaine décennie."
}
```

> Finished chain.
Complex translation result: {'input': 'Can you translate this technical sentence to French: "Artificial intelligence will revolutionize healthcare in the next decade."', 'output': 'L’intelligence artificielle révolutionnera les soins de santé au cours de la prochaine décennie.'}


Example 4 - Language guess + translate

In [10]:
# Example: Guess the language and translate to English
query_language_guess_and_translate = {'input': 'Can you guess what language this is and translate it to English: "Hola, ¿cómo estás?"'}

# Invoke the tool to guess language and translate
try:
    response_language_guess_and_translate = agent_executor.invoke(query_language_guess_and_translate)
    print("Guess and Translate result:", response_language_guess_and_translate)
except Exception as e:
    print(f"Error during language guess and translation: {e}")




> Entering new AgentExecutor chain...
Thought: The phrase "Hola, ¿cómo estás?" appears to be in Spanish. I will use the Azure Translator tool to confirm and translate it to English.

Action:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Hola, ¿cómo estás?",
    "to_language": "en"
  }
}
```Hello, how are you?Action:
```
{
  "action": "Final Answer",
  "action_input": "The phrase 'Hola, ¿cómo estás?' is in Spanish and translates to 'Hello, how are you?' in English."
}
```

> Finished chain.
Guess and Translate result: {'input': 'Can you guess what language this is and translate it to English: "Hola, ¿cómo estás?"', 'output': "The phrase 'Hola, ¿cómo estás?' is in Spanish and translates to 'Hello, how are you?' in English."}


Example 5 - Translate to Multiple Languages

In [11]:
# Example: Translate "Good morning" into Spanish, French, and Italian
query_multiple_translations = {'input': 'Can you translate "Good morning" into Spanish, French, and Italian?'}

# Invoke the tool to translate into multiple languages
try:
    response_multiple_translations = agent_executor.invoke(query_multiple_translations)
    print("Multiple translations result:", response_multiple_translations)
except Exception as e:
    print(f"Error during translation to multiple languages: {e}")




> Entering new AgentExecutor chain...
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Good morning",
    "to_language": "es"
  }
}Buenos díasAction:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Good morning",
    "to_language": "fr"
  }
}
```BonjourAction:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Good morning",
    "to_language": "it"
  }
}
```BuongiornoAction:
```
{
  "action": "Final Answer",
  "action_input": "The translation of 'Good morning' is 'Buenos días' in Spanish, 'Bonjour' in French, and 'Buongiorno' in Italian."
}
```

> Finished chain.
Multiple translations result: {'input': 'Can you translate "Good morning" into Spanish, French, and Italian?', 'output': "The translation of 'Good morning' is 'Buenos días' in Spanish, 'Bonjour' in French, and 'Buongiorno' in Italian."}


Example 6 - Asking for Idiomatic Expressions

In [12]:
# Example: Translate the idiom "It's raining cats and dogs" into Spanish
query_idiom_translation = {'input': 'Can you translate the idiom "It\'s raining cats and dogs" into Spanish?'}

# Invoke the tool to translate the idiom
try:
    response_idiom_translation = agent_executor.invoke(query_idiom_translation)
    print("Idiom translation result:", response_idiom_translation)
except Exception as e:
    print(f"Error during idiom translation: {e}")




> Entering new AgentExecutor chain...
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "It's raining cats and dogs",
    "to_language": "es"
  }
}Está lloviendo a cántaros.{
  "action": "Final Answer",
  "action_input": "The idiom 'It's raining cats and dogs' translates to Spanish as 'Está lloviendo a cántaros.'"
}

> Finished chain.
Idiom translation result: {'input': 'Can you translate the idiom "It\'s raining cats and dogs" into Spanish?', 'output': "The idiom 'It's raining cats and dogs' translates to Spanish as 'Está lloviendo a cántaros.'"}


Example 7 - Asking to Translate Back and Forth

In [13]:
# Example: Translate a phrase to German and then back to English
query_back_and_forth = {'input': 'Can you translate "The quick brown fox jumps over the lazy dog" into German, and then back to English?'}

# Invoke the tool to translate back and forth
try:
    response_back_and_forth = agent_executor.invoke(query_back_and_forth)
    print("Back-and-forth translation result:", response_back_and_forth)
except Exception as e:
    print(f"Error during back-and-forth translation: {e}")




> Entering new AgentExecutor chain...
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "The quick brown fox jumps over the lazy dog",
    "to_language": "de"
  }
}Der flinke braune Fuchs springt über den faulen Hund{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Der flinke braune Fuchs springt über den faulen Hund",
    "to_language": "en"
  }
}The nimble brown fox jumps over the lazy dogAction:
```
{
  "action": "Final Answer",
  "action_input": "The translation of 'The quick brown fox jumps over the lazy dog' into German is 'Der flinke braune Fuchs springt über den faulen Hund'. When translated back to English, it becomes 'The nimble brown fox jumps over the lazy dog'."
}
```

> Finished chain.
Back-and-forth translation result: {'input': 'Can you translate "The quick brown fox jumps over the lazy dog" into German, and then back to English?', 'output': "The translation of 'The quick brown fox jumps over the lazy dog' into German is 'D

Example 8 - Translating Phrases with Special Characters or Emojis

In [14]:
# Example: Translate a phrase with text and emojis into French
query_with_text_and_emojis = {'input': 'Can you translate this phrase into French: "I\'m happy 😊 because it\'s sunny ☀️ today."'}

# Invoke the tool to translate the phrase with text and emojis
try:
    response_with_text_and_emojis = agent_executor.invoke(query_with_text_and_emojis)
    print("Emoji translation result:", response_with_text_and_emojis)
except Exception as e:
    print(f"Error during emoji translation: {e}")





> Entering new AgentExecutor chain...
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "I'm happy 😊 because it's sunny ☀️ today.",
    "to_language": "fr"
  }
}Je suis content 😊 parce qu’il fait beau ☀️ aujourd’hui.{
  "action": "Final Answer",
  "action_input": "Je suis content 😊 parce qu’il fait beau ☀️ aujourd’hui."
}

> Finished chain.
Emoji translation result: {'input': 'Can you translate this phrase into French: "I\'m happy 😊 because it\'s sunny ☀️ today."', 'output': 'Je suis content 😊 parce qu’il fait beau ☀️ aujourd’hui.'}


Example 9 - Longer Paragraph Translation

In [15]:
# Example: Translate a longer paragraph into Spanish
query_paragraph_translation = {'input': 'Can you translate this paragraph to Spanish: "Machine learning is a subset of artificial intelligence, and it allows computers to learn from data without being explicitly programmed."'}

# Invoke the tool to translate a paragraph
try:
    response_paragraph_translation = agent_executor.invoke(query_paragraph_translation)
    print("Paragraph translation result:", response_paragraph_translation)
except Exception as e:
    print(f"Error during paragraph translation: {e}")




> Entering new AgentExecutor chain...
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Machine learning is a subset of artificial intelligence, and it allows computers to learn from data without being explicitly programmed.",
    "to_language": "es"
  }
}El aprendizaje automático es un subconjunto de la inteligencia artificial y permite a los ordenadores aprender de los datos sin necesidad de programarlos explícitamente.{
  "action": "Final Answer",
  "action_input": "El aprendizaje automático es un subconjunto de la inteligencia artificial y permite a los ordenadores aprender de los datos sin necesidad de programarlos explícitamente."
}

> Finished chain.
Paragraph translation result: {'input': 'Can you translate this paragraph to Spanish: "Machine learning is a subset of artificial intelligence, and it allows computers to learn from data without being explicitly programmed."', 'output': 'El aprendizaje automático es un subconjunto de la inteligencia artifici

Example 10 - Asking for a Reverse Translation

In [16]:
# Example: Reverse translate a complex sentence through multiple languages (English -> German -> Spanish -> English)
query_reverse_translation_complex = {
    'input': 'Translate "The quick brown fox jumps over the lazy dog" into German, then into Spanish, and finally back to English. Let\'s see if the meaning remains the same.'
}

# Invoke the tool for complex reverse translation
try:
    response_reverse_translation_complex = agent_executor.invoke(query_reverse_translation_complex)
    print("Complex reverse translation result:", response_reverse_translation_complex)
except Exception as e:
    print(f"Error during reverse translation: {e}")





> Entering new AgentExecutor chain...
Thought: I will first translate the sentence into German, then into Spanish, and finally back to English.

Action:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "The quick brown fox jumps over the lazy dog",
    "to_language": "de"
  }
}
```Der flinke braune Fuchs springt über den faulen HundAction:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Der flinke braune Fuchs springt über den faulen Hund",
    "to_language": "es"
  }
}
```El ágil zorro marrón salta sobre el perro perezosoAction:
```
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "El ágil zorro marrón salta sobre el perro perezoso",
    "to_language": "en"
  }
}
```The agile brown fox jumps on the lazy dogThe meaning of the sentence has slightly changed during the translations, particularly the preposition "over" has been altered to "on."

Action:
```
{
  "action": "Final Answer",
  "action_input": "The 